# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The dataset accounts for 17 campaigns.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import RandomizedSearchCV, HalvingRandomSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectFromModel

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

### Business Objective
To develop a classification model that predicts whether a client will subscribe to a term deposit. This model will be used to create marketing campaigns aimed at customers with a high probability of subscribing to a term deposit.

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [5]:
drop_cols_baseline = ['campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']
X = df.drop(columns = drop_cols_baseline)
y = df['y']

In [6]:
X.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307


In [7]:
transformer = make_column_transformer(
    (OneHotEncoder(drop='if_binary'), ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week']),
    remainder=StandardScaler(),
)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

A: A good baseline to beat is the Majority Class Baseline. In this case the majority class is 'no' which is 88.7% of the dataset. A model with accuracy greater than 88.7% would be ideal.

In [9]:
(y.value_counts().max() / len(y)) * 100

np.float64(88.73458288821988)

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [10]:
lgr_pipe = Pipeline([
    ('transformer', transformer),
    ('lgr', LogisticRegression()),
]).fit(X_train, y_train)

### Problem 9: Score the Model

What is the accuracy of your model?

In [11]:
lgr_pipe.score(X_test, y_test)

0.9014323865015781

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

### Logistic Regression

In [12]:
t1 = time.time()
lgr_pipe = Pipeline([
    ('transformer', transformer),
    ('lgr', LogisticRegression()),
]).fit(X_train, y_train)

lgr_default_train_time = time.time() - t1
lgr_default_train_score = lgr_pipe.score(X_train, y_train)
lgr_default_test_score = lgr_pipe.score(X_test, y_test)

print("Logistic Regression")
print(f"Train Accuracy: {lgr_default_train_score}")
print(f"Test Accuracy: {lgr_default_test_score}")

Logistic Regression
Train Accuracy: 0.8988770864946889
Test Accuracy: 0.9014323865015781


### K-Nearest Neighbors

In [13]:
t1 = time.time()
knn_pipe = Pipeline([
    ('transformer', transformer),
    ('knn', KNeighborsClassifier()),
]).fit(X_train, y_train)

knn_default_train_time = time.time() - t1
knn_default_train_score = knn_pipe.score(X_train, y_train)
knn_default_test_score = knn_pipe.score(X_test, y_test)

print("K-Nearest Neighbors")
print(f"Train Accuracy: {knn_default_train_score}")
print(f"Test Accuracy: {knn_default_test_score}")

K-Nearest Neighbors
Train Accuracy: 0.9166919575113809
Test Accuracy: 0.8932993445010925


In [14]:
t1 = time.time()
dtree_pipe = Pipeline([
    ('transformer', transformer),
    ('dtree', DecisionTreeClassifier()),
]).fit(X_train, y_train)

dtree_default_train_time = time.time() - t1
dtree_default_train_score = dtree_pipe.score(X_train, y_train)
dtree_default_test_score = dtree_pipe.score(X_test, y_test)

print("Decision Tree")
print(f"Train Accuracy: {dtree_default_train_score}")
print(f"Test Accuracy: {dtree_default_test_score}")

Decision Tree
Train Accuracy: 0.9999696509863429
Test Accuracy: 0.8623452294246177


In [15]:
t1 = time.time()
svc_pipe = Pipeline([
    ('transformer', transformer),
    ('svc', SVC()),
]).fit(X_train, y_train)

svc_default_train_time = time.time() - t1
svc_default_train_score = svc_pipe.score(X_train, y_train)
svc_default_test_score = svc_pipe.score(X_test, y_test)

print("Support Vector Machine")
print(f"Train Accuracy: {svc_default_train_score}")
print(f"Test Accuracy: {svc_default_test_score}")

Support Vector Machine
Train Accuracy: 0.9124430955993931
Test Accuracy: 0.9031318281136198


In [16]:
model_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'Decision Tree', 'SVC'],
    'Train Time': [lgr_default_train_time, knn_default_train_time, dtree_default_train_time, svc_default_train_time],
    'Train Accuracy': [lgr_default_train_score, knn_default_train_score, dtree_default_train_score, svc_default_train_score],
    'Test Accuracy': [lgr_default_test_score, knn_default_test_score, dtree_default_test_score, svc_default_test_score],
})
model_df

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.150093,0.898877,0.901432
1,KNN,0.097410,0.916692,0.893299
2,Decision Tree,0.580995,0.999970,0.862345
3,SVC,11.639476,0.912443,0.903132


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

### Evaluation
In order to evaluate all models fairly, I choose to use `permutation_importance` on all models.

In [17]:
drop_cols_baseline = ['campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'duration']
X = df.drop(columns = drop_cols_baseline)
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Logistic Regression

In [18]:
t1 = time.time()
lgr_pipe = Pipeline([
    ('transformer', transformer),
    ('lgr', LogisticRegression(multi_class='multinomial')),
])

grid_lgr = RandomizedSearchCV(
    lgr_pipe,
    {
        'lgr__C': [0.01, 0.1, 1, 10, 100],
        'lgr__penalty': ['l1', 'l2', 'elasticnet'],
        'lgr__solver': ['liblinear', 'saga'],
        'lgr__max_iter': [100, 200, 500, 1000],
    },
    cv = 5,
).fit(X, y)


lgr_train_time = time.time() - t1
lgr_train_accuracy = grid_lgr.score(X_train, y_train)
lgr_test_accuracy = grid_lgr.score(X_test, y_test)

print("Logistic Regression")
print(f"Train Accuracy: {lgr_train_accuracy}")
print(f"Test Accuracy: {lgr_test_accuracy}")

Logistic Regression
Train Accuracy: 0.887556904400607
Test Accuracy: 0.8863801893663511


### Logistic Regression: Permutation Importance

In [19]:
result_lgr = permutation_importance(grid_lgr, X_test, y_test)
df_perm_lgr = pd.DataFrame(result_lgr.importances.T, columns=X_test.columns)
df_perm_lgr.mean().sort_values(ascending=False)

education      0.000000
housing        0.000000
loan           0.000000
contact       -0.000024
day_of_week   -0.000024
marital       -0.000049
default       -0.000049
age           -0.000121
job           -0.000121
month         -0.000121
dtype: float64

Can adding polynomial complexity increase an already good Logistic Regression test score?

In [20]:
t1 = time.time()
lgr_poly_pipe = Pipeline([
    ('transformer', transformer),
    ('poly', PolynomialFeatures()),
    ('lgr', LogisticRegression(multi_class='multinomial')),
])

grid_poly_lgr = RandomizedSearchCV(
    lgr_poly_pipe,
    {
        'lgr__C': [0.01, 0.1, 1, 10, 100],
        'lgr__penalty': ['l1', 'l2', 'elasticnet'],
        'lgr__fit_intercept': [True, False],
        'lgr__max_iter': [100, 200, 500, 1000],
        'poly__degree': [2, 3],
    },
    cv = 5,
).fit(X, y)


lgr_poly_train_time = time.time() - t1
lgr_poly_train_accuracy = grid_poly_lgr.score(X_train, y_train)
lgr_poly_test_accuracy = grid_poly_lgr.score(X_test, y_test)

print("Logistic Regression with Polynomial Features")
print(f"Train Accuracy: {lgr_poly_train_accuracy}")
print(f"Test Accuracy: {lgr_poly_test_accuracy}")

Logistic Regression with Polynomial Features
Train Accuracy: 0.8896206373292868
Test Accuracy: 0.8894149065307113


### Logistic Regression w/ Polynomial Feature: Permutation Importance
Top fields:
- `month`
- `age`
- `contact`

In [21]:
result_lgr_poly = permutation_importance(grid_poly_lgr, X_test, y_test)
df_perm_lgr_poly = pd.DataFrame(result_lgr_poly.importances.T, columns=X_test.columns)
df_perm_lgr_poly.mean().sort_values(ascending=False)

month          0.005632
age            0.002064
contact        0.000971
default        0.000607
education      0.000534
job            0.000437
loan           0.000316
day_of_week    0.000291
housing       -0.000388
marital       -0.000413
dtype: float64

### K-Nearest Neighbors

In [22]:
t1 = time.time()
knn_pipe = Pipeline([
    ('transformer', transformer),
    ('knn', KNeighborsClassifier()),
]).fit(X_train, y_train)

grid_knn = RandomizedSearchCV(
    knn_pipe,
    {
        'knn__n_neighbors': range(1, 21, 2),
        'knn__weights': ['uniform', 'distance'],
        'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'knn__leaf_size': range(10, 50, 10),
        'knn__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
        'knn__p': [1, 2]
    },
    cv = 5,
).fit(X, y)

knn_train_time = time.time() - t1
knn_train_accuracy = grid_knn.score(X_train, y_train)
knn_test_accuracy = grid_knn.score(X_test, y_test)

print("K-Nearest Neighbors")
print(f"Train Accuracy: {knn_train_accuracy}")
print(f"Test Accuracy: {knn_test_accuracy}")

K-Nearest Neighbors
Train Accuracy: 0.891350531107739
Test Accuracy: 0.8912357368293276


### K-Nearest Neighbors: Permutation Importance
Top fields:
- `age`
- `month`
- `job`

In [23]:
result_knn = permutation_importance(grid_knn, X_test, y_test)
df_perm_knn = pd.DataFrame(result_knn.importances.T, columns=X_test.columns)
df_perm_knn.mean().sort_values(ascending=False)

age            0.005171
month          0.003617
job            0.001942
day_of_week    0.001772
contact        0.001432
education      0.001044
default        0.000801
housing        0.000388
marital        0.000097
loan          -0.000486
dtype: float64

### Decision Tree

In [24]:
t1 = time.time()
dtree_pipe = Pipeline([
    ('transformer', transformer),
    ('dtree', DecisionTreeClassifier()),
]).fit(X_train, y_train)

grid_dtree = RandomizedSearchCV(
    dtree_pipe,
    {
        'dtree__criterion': ['gini', 'entropy', 'log_loss'],
        'dtree__splitter': ['best', 'random'],
        'dtree__max_depth': [None, 5, 10, 15, 20, 25, 50, 100],
        'dtree__min_samples_split': [1, 2, 5, 10, 20, 30, 100],
        'dtree__min_samples_leaf': [1, 2, 4, 6, 10, 20],
    },
    cv = 5,
).fit(X, y)

dtree_train_time = time.time() - t1
dtree_train_accuracy = grid_dtree.score(X_train, y_train)
dtree_test_accuracy = grid_dtree.score(X_test, y_test)

print("Decision Tree")
print(f"Train Accuracy: {dtree_train_accuracy}")
print(f"Test Accuracy: {dtree_test_accuracy}")

Decision Tree
Train Accuracy: 0.8950227617602428
Test Accuracy: 0.8931779558145181


### Decision Tree: Permutation Importance
Top fields:
- `month`
- `contact`
- `age`

In [25]:
result_dtree = permutation_importance(grid_dtree, X_test, y_test)
df_perm_dtree = pd.DataFrame(result_dtree.importances.T, columns=X_test.columns)
df_perm_dtree.mean().sort_values(ascending=False)

month          0.040155
contact        0.029667
age            0.008861
day_of_week    0.004443
job            0.004176
education      0.003059
default        0.001845
marital        0.001360
housing        0.001335
loan           0.000753
dtype: float64

### Support Vector Machine

In [26]:
t1 = time.time()
svc_pipe = Pipeline([
    ('transformer', transformer),
    ('svc', SVC()),
]).fit(X_train, y_train)

grid_svc = RandomizedSearchCV(
    svc_pipe,
    {
        'svc__C': [0.1, 1, 10, 100, 1000],
        'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'svc__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
        'svc__degree': [1, 2, 3, 4, 5, 6, 7, 8, 9 ,10],
        'svc__decision_function_shape': ['ovr', 'ovo'],
        'svc__max_iter': [10, 100, 1000, 5000, 10_000],
    },
    cv = 5,
).fit(X, y)

svc_train_time = time.time() - t1
svc_train_accuracy = grid_svc.score(X_train, y_train)
svc_test_accuracy = grid_svc.score(X_test, y_test)

print("Support Vector Machine")
print(f"Train Accuracy: {svc_train_accuracy}")
print(f"Test Accuracy: {svc_test_accuracy}")

Support Vector Machine
Train Accuracy: 0.8298027314112292
Test Accuracy: 0.8284777858703569


### Support Vector Machine: Permutation Importance
Top fields:
- `age`
- `job`
- `marital`

In [27]:
result_svc = permutation_importance(grid_svc, X_test, y_test)
df_perm_svc = pd.DataFrame(result_svc.importances.T, columns=X_test.columns)
df_perm_svc.mean().sort_values(ascending=False)

age            0.027871
marital        0.004151
month          0.001481
education      0.000995
contact        0.000947
day_of_week    0.000753
loan           0.000753
job           -0.000121
default       -0.000825
housing       -0.001020
dtype: float64

### Findings

In [28]:
model_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'Logistic Regression w/ Poly', 'KNN', 'Decision Tree', 'SVC'],
    'Train Time': [lgr_train_time, lgr_poly_train_time, knn_train_time, dtree_train_time, svc_train_time],
    'Train Accuracy': [lgr_train_accuracy, lgr_poly_train_accuracy, knn_train_accuracy, dtree_train_accuracy, svc_train_accuracy],
    'Test Accuracy': [lgr_test_accuracy, lgr_poly_test_accuracy, knn_test_accuracy, dtree_test_accuracy, svc_test_accuracy],
})
model_df

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,51.273101,0.887557,0.886380
1,Logistic Regression w/ Poly,324.079487,0.889621,0.889415
2,KNN,146.915529,0.891351,0.891236
3,Decision Tree,15.044021,0.895023,0.893178
4,SVC,218.800592,0.829803,0.828478


We see from the data above, the accuracy for most of these classifiers are pretty even. 

### Production Model Selection
The metric we used for scoring was accuracy. As this is a highly imbalanced data set using this may not be the best metric to base performance off of as 87% of the dataset belongs to one class of data. In order to more correctly identify the metric required the Business Objective would need to be more defined. Would the client want to air on the side of Misclassifying a Positive instance (Precision) or perfectly classifying the positive instances (Recall).

Based on this question as we selected accuracy as our metric. I would select Decision Tree Classifier as the model to deploy in Production. As this has the best accuracy, and the fastest train time. I had to switch to RandomizedSearchCV as GridSearchCV was taking far too long to train. In a production environment I would take the extra time to train with GridSearchCV.

### Feature Importance
The features that most influenced the Decision Tree Classifiers classification was as follows:

- `month`
- `contact`
- `age`

In [31]:
fig = px.bar(df_perm_dtree.mean().sort_values(ascending=False), title='Permutation Importance', labels={'index': 'Feature', 'value': 'Importance'})
fig.show()

### Recommendations

In [32]:
df2 = df.copy()
df2['y_enc'] = df2['y'].apply(lambda x: 1 if x == 'yes' else 0)
df2[['month', 'contact', 'age', 'y_enc']].groupby(['month', 'contact', 'age']).sum().sort_values(by='y_enc', ascending=False)

y_enc
month contact   age       
may   cellular  33      43
                31      41
                28      35
jul   cellular  31      33
aug   cellular  31      32
...                    ...
dec   telephone 48       0
                47       0
                42       0
oct   cellular  43       0
apr   cellular  18       0

[1083 rows x 1 columns]

In the table above, I grouped the original dataframe by the 3 most influencial features according to the Decision Tree Classifier model and summed the y values to count how many 'yes's for each grouping. According to this grouping the most yes's are in the month of May where the contact type is cellular and the age is over the age of 30. This is what I would recommend to the bank.

In [33]:
df['contact'].value_counts()

contact
cellular     26144
telephone    15044
Name: count, dtype: int64

In [34]:
df['age'].value_counts()

age
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
89       2
91       2
94       1
87       1
95       1
Name: count, Length: 78, dtype: int64

### Future considerations
Looking at the `age` and `contact` fields: 

`contact`:
- for this field I may think about removing this to see how it would affect the accuracy of the model. Especially as telephones are becoming less and less common.

`age`:
- there are 78 different values for age which could skew some results. We'd probably get some more correlation if we grouped the age into bins.

Taking these things into consideration I'd attempt to re-train the model changing one, then the other, then both and comparing the results and accuracy of each.

##### Questions

A question I'd like to ask in the future is about the `duration` feature. I understand high correlation to no when the value is zero or an unreasonable length. If those unreasonable values are dropped is there a still a high correlation to 'yes' or 'no'? I also understand that you can't know the duration before a phone call, but if there is still a highly correlated value perhaps it could share insight on how long you should try to keep this customer on the line for the best chances. If we loop through some call duration values keeping everything else the same it could increase the chances of a 'yes'.